In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
data = pd.read_csv("../data/test_df_en.csv", sep=";")
data = data.drop(data.columns[0], axis=1)

In [8]:
# Create dictionary and corpus
dictionary = corpora.Dictionary(data['selected'])
corpus = [dictionary.doc2bow(text) for text in data['selected']]

# Apply LDA model
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

# Print the topics
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# Get the topic distribution for each document
topic_distribution = lda_model[corpus]

# Add topic distribution to DataFrame
data['topic_distribution'] = topic_distribution

Topic: 0 
Words: 0.034*"high" + 0.015*"ah" + 0.013*"little" + 0.012*"everybody" + 0.010*"wild" + 0.009*"cause" + 0.008*"die" + 0.008*"sweet" + 0.007*"music" + 0.007*"ready"
Topic: 1 
Words: 0.034*"baby" + 0.020*"girl" + 0.013*"good" + 0.013*"wanna" + 0.010*"man" + 0.010*"cause" + 0.008*"real" + 0.008*"slow" + 0.008*"right" + 0.007*"love"
Topic: 2 
Words: 0.010*"life" + 0.007*"world" + 0.007*"many" + 0.007*"black" + 0.006*"call" + 0.006*"blood" + 0.006*"things" + 0.005*"live" + 0.005*"sun" + 0.004*"house"
Topic: 3 
Words: 0.026*"time" + 0.017*"love" + 0.016*"way" + 0.013*"life" + 0.013*"heart" + 0.010*"gonna" + 0.010*"night" + 0.010*"cause" + 0.009*"day" + 0.008*"eyes"
Topic: 4 
Words: 0.011*"niggas" + 0.010*"nigga" + 0.010*"shit" + 0.008*"word" + 0.008*"hip" + 0.008*"cause" + 0.007*"jump" + 0.007*"hop" + 0.006*"girls" + 0.006*"man"


In [6]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

stop_words = set(stopwords.words('english'))
stop_words.update(['I', 'aye', 'oh', 'ho', 'uh', 'yeah','wa','hey','da','duh','ooh','ohh',
                   'la','yo','ya','na'])
def filter_adjectives_nouns(text):
    # Tokenize le texte
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    words = [word.lower() for word in words if word.lower() not in stop_words ]

    pos_tags = pos_tag(words)

    adjectives_nouns = [word for word, pos in pos_tags if pos.startswith('NN') or pos.startswith('JJ')]

    return adjectives_nouns

In [7]:
data['selected'] = data['lyrics'].apply(filter_adjectives_nouns)

In [24]:
data

,spotify_id,name,artist,position,genre_name,lyrics,language,selected,topic_distribution
0,3B7i9OKRRmIsSBHEbJz58Y,Grind With Me,Pretty Ricky,10.0,pop rap,"Ohh Ooh-oh, yeah Hey, hey, hey, yeah Baby, g...",en,"[baby, grind, relax, take, time, deeper, ride,...","[(1, 0.9972256)]"
1,0kPgn96FXEO9Bnz9QN1Tc0,We In Da City,Young Dro,18.0,pop rap,Yeah yeah Yeah yeah Yeah yeah Yeah yeah Young...,en,"[young, dro, turn, car, roof, car, roof, city,...","[(1, 0.50212723), (4, 0.493136)]"
2,6Y9kdB2O0h9gq9y2vclsWT,Shorty Wanna Ride,Young Buck,22.0,pop rap,Shorty wanna ride with me (Ride with me) Shor...,en,"[shorty, wanna, ride, shorty, wanna, ride, rid...","[(1, 0.89551055), (4, 0.10248195)]"
3,0svl7cK07gK1ia5ainczU5,Break Ya Neck,Busta Rhymes,28.0,pop rap,"Yeah, check it out, see The only thing you ne...",en,"[see, thing, right, nod, fuckin, head, break, ...","[(0, 0.673072), (4, 0.32480398)]"
4,1DFPibRb9RU3yfM3wOLdDT,Tru,Lloyd,32.0,pop rap,"I lost it all, my friends, my loved ones But ...",en,"[friends, ones, life, chance, grow, struggle, ...","[(3, 0.9922578)]"
...,...,...,...,...,...,...,...,...,...
10301,35kAc40pbM9VpzQvnNMAZO,Raw Shit,Jaylib,102.0,east coast hip hop,"Uh Feel it (Yeah, yeah) Check it now What'd y...",en,"[feel, check, come, c, c, kweli, jaylib, colla...","[(0, 0.06392147), (1, 0.113179736), (2, 0.0216..."
10302,46ePNyvaDBKsxusDXDj7Wg,Wipeout,Fat Boys,114.0,east coast hip hop,"Hehehehehehee, wipeout Wiping out, wipeout W...",en,"[hehehehehehee, wipeout, wah, wah, wah, wipeou...","[(0, 0.99452436)]"
10303,6msTUCqIfVOWe7pzWGzh6F,Leaving the Past,Immortal Technique,118.0,east coast hip hop,"They told me I would never make it, I would n...",en,"[reality, nourishment, people, guess, hard, st...","[(0, 0.8551019), (1, 0.14188409)]"
10304,5o7zlUWDdRlunP9S0OTm9R,Crumbs On the Table,D-Nice,123.0,east coast hip hop,"(Is that a turntable? Well get on it, it's you...",en,"[turntable, turn, chicken, egg, mc, renegade, ...","[(2, 0.20921499), (3, 0.13533655), (4, 0.65305..."


In [28]:
# Create a dictionary to store the distribution of topics for each genre
genre_topic_distribution = {}

# Iterate over each item in the topic_genre_associations dictionary
for index, (topic_index, genre_name) in topic_genre_associations.items():
    # If the genre_name is not already in the genre_topic_distribution dictionary, add it with an empty list
    if genre_name not in genre_topic_distribution:
        genre_topic_distribution[genre_name] = [0] * lda_model.num_topics
    
    # Increment the count for the topic associated with the current genre
    genre_topic_distribution[genre_name][topic_index] += 1

# Print the distribution of topics for each genre
for genre_name, topic_distribution in genre_topic_distribution.items():
    print(f"Genre: {genre_name}")
    for topic_index, count in enumerate(topic_distribution):
        print(f"Topic {topic_index}: {count}")


Genre: pop rap
Topic 0: 9
Topic 1: 17
Topic 2: 0
Topic 3: 16
Topic 4: 18
Genre: pop
Topic 0: 10
Topic 1: 23
Topic 2: 5
Topic 3: 65
Topic 4: 4
Genre: post-teen pop
Topic 0: 11
Topic 1: 25
Topic 2: 3
Topic 3: 78
Topic 4: 4
Genre: trap music
Topic 0: 6
Topic 1: 19
Topic 2: 3
Topic 3: 5
Topic 4: 23
Genre: dance pop
Topic 0: 11
Topic 1: 27
Topic 2: 2
Topic 3: 53
Topic 4: 4
Genre: rock
Topic 0: 12
Topic 1: 17
Topic 2: 16
Topic 3: 55
Topic 4: 9
Genre: rap
Topic 0: 6
Topic 1: 20
Topic 2: 1
Topic 3: 5
Topic 4: 25
Genre: tropical house
Topic 0: 4
Topic 1: 4
Topic 2: 0
Topic 3: 26
Topic 4: 1
Genre: neo mellow
Topic 0: 6
Topic 1: 4
Topic 2: 3
Topic 3: 115
Topic 4: 0
Genre: southern hip hop
Topic 0: 6
Topic 1: 15
Topic 2: 3
Topic 3: 7
Topic 4: 18
Genre: edm
Topic 0: 5
Topic 1: 1
Topic 2: 1
Topic 3: 14
Topic 4: 1
Genre: dwn trap
Topic 0: 4
Topic 1: 24
Topic 2: 1
Topic 3: 6
Topic 4: 22
Genre: hip hop
Topic 0: 12
Topic 1: 19
Topic 2: 9
Topic 3: 6
Topic 4: 29
Genre: latin
Topic 0: 0
Topic 1: 1
Topic 2:

In [27]:
# Create a dictionary to store the associations between genre_name and the topic with the maximum probability
topic_genre_associations = {}

# Iterate over each row in the DataFrame
for index, row in data.iterrows():
    # Get the topic with the maximum probability
    max_topic = max(row['topic_distribution'], key=lambda x: x[1])
    
    # Extract the topic index and probability
    max_topic_index, max_topic_prob = max_topic
    
    # Get the corresponding genre_name
    genre_name = row['genre_name']
    
    # Associate the genre_name with the topic index
    topic_genre_associations[index] = (max_topic_index, genre_name)


topic_genre_associations

{0: (1, 'pop rap'),
 1: (1, 'pop rap'),
 2: (1, 'pop rap'),
 3: (0, 'pop rap'),
 4: (3, 'pop rap'),
 5: (4, 'pop rap'),
 6: (0, 'pop rap'),
 7: (0, 'pop rap'),
 8: (3, 'pop rap'),
 9: (4, 'pop rap'),
 10: (1, 'pop rap'),
 11: (1, 'pop rap'),
 12: (3, 'pop rap'),
 13: (1, 'pop rap'),
 14: (4, 'pop rap'),
 15: (0, 'pop rap'),
 16: (4, 'pop rap'),
 17: (3, 'pop rap'),
 18: (3, 'pop rap'),
 19: (4, 'pop rap'),
 20: (0, 'pop rap'),
 21: (4, 'pop rap'),
 22: (4, 'pop rap'),
 23: (3, 'pop rap'),
 24: (1, 'pop rap'),
 25: (3, 'pop rap'),
 26: (0, 'pop rap'),
 27: (4, 'pop rap'),
 28: (4, 'pop rap'),
 29: (1, 'pop rap'),
 30: (3, 'pop rap'),
 31: (1, 'pop rap'),
 32: (4, 'pop rap'),
 33: (1, 'pop rap'),
 34: (0, 'pop rap'),
 35: (3, 'pop rap'),
 36: (0, 'pop rap'),
 37: (3, 'pop rap'),
 38: (1, 'pop rap'),
 39: (1, 'pop rap'),
 40: (3, 'pop rap'),
 41: (4, 'pop rap'),
 42: (4, 'pop rap'),
 43: (1, 'pop rap'),
 44: (3, 'pop rap'),
 45: (4, 'pop rap'),
 46: (4, 'pop rap'),
 47: (1, 'pop rap'),
 4

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Assuming 'lda_model' is your trained LDA model
topics = lda_model.show_topics(formatted=False)

# Plot word clouds for each topic
for topic_id, words in topics:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(words))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Topic {topic_id}')
    plt.axis('off')
    plt.show()


In [11]:
from gensim import corpora, models
import pyLDAvis.gensim

# Assuming you have trained an LDA model named "ldamodel"
# and you have the corpus and dictionary used for training the model

# Visualize the topics
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)
lda_display.topic_info['genre_name'] = topic_genre_df['Most Prevalent Genre']
pyLDAvis.display(lda_display)

NameError: name 'topic_genre_df' is not defined